# Welcome to OSMNxMapping ☀️!

_Chapter 8_ will dive into an advanced pipelining workflow with the `UrbanPipeline` class from OSMNxMapping using multi enrichments as seen in chapter 7, yet with a novelty: multi mapping. We want to map more than one pair of latitude and longitude, then multi-enrich either each pair once or more than once.

We'll therefore process taxi trip data from New York City, mapping pickup and dropoff pair locations to a Downtown Brooklyn street network and enriching it with multiple metrics: pickup counts and dropoff counts per node.

**Goals**:
- Import the OSMNxMapping library and required modules.
- Build an `UrbanPipeline` with steps for loading, preprocessing, multiple mappings, and multiple enrichments.
- Execute the pipeline using `compose_transform`.
- Visualise the enriched network with multiple attributes.
- Understand how to handle multiple mappings and enrichments seamlessly.

Unlike previous notebooks, we won’t use Auctus here—data must be available locally in CSV, Shapefile, or Parquet format. We'll use a sample CSV file (`taxis.csv`). For foundational steps or alternative approaches, refer to the chapter's 1 notebook.

Let’s dive in! 🚀

## Step 1: Import the Library and Modules

We import the `osmnx_mapping` library and necessary pipeline modules to construct our workflow.

In [ ]:
import osmnx_mapping as oxm
from osmnx_mapping import UrbanPipeline, CreateNetwork, CSVLoader, CreatePreprocessor, CreateEnricher, InteractiveVisualiser, StaticVisualiser

## Step 2: Define and Execute the Urban Pipeline

We define an `UrbanPipeline` to process taxi trip data and enrich the network:

- **Loader**: Loads taxi data from CSV.
- **Impute Pickup/Dropoff**: Ensures valid coordinates.
- **Filter**: Retains data within the network’s bounding box.
- **Network**: Maps pickup and dropoff locations to nodes.
- **Enrich Pickup/Dropoff**: Counts pickups and dropoffs per node.

We execute it with `compose_transform`.

> **Note**: Adjust the file path if needed and ensure column names match your dataset.


> **IMPORTANT**: Notice that here we will as much as possible specify `latitude_column_name` and `longitude_column_name` to avoid confusing ourselves because here we are in more depth than only one pair of latitude/longitude columns we are in two. Imagine if N. We therefore specify as much as possible so that we can distinguish each step –––– Pssstt! Not only you but people re utilising your saved and shared pipeline :)

In [ ]:
pipeline = UrbanPipeline([
    ('loader', CSVLoader(file_path='./../data/TAXIS/csv/taxisvis1M.csv')),
    ('network', CreateNetwork()
        .with_place('Downtown, Brooklyn, New York, USA', network_type='drive')
        .with_mapping(mapping_type='node', longitude_column_name='pickup_longitude', latitude_column_name='pickup_latitude', output_column='pickup_node')
        .with_mapping(mapping_type='node', longitude_column_name='dropoff_longitude', latitude_column_name='dropoff_latitude', output_column='dropoff_node')
        .build()
    ),
    ('impute_pickup', CreatePreprocessor()
         .with_imputer(
            imputer_type='SimpleGeoImputer',
            latitude_column_name='pickup_latitude',
            longitude_column_name='pickup_longitude'
        ).build()
    ),
    ('impute_dropoff', CreatePreprocessor()
         .with_imputer(
        imputer_type='SimpleGeoImputer',
            latitude_column_name='dropoff_latitude',
            longitude_column_name='dropoff_longitude'
        ).build()
    ),
    ('filter', CreatePreprocessor()
         .with_filter(
            filter_type='BoundingBoxFilter'
        ).build()
    ),
    ('enrich_pickup', CreateEnricher()
        .with_data(group_by='pickup_node')
        .count_by(target='nodes', output_column='pickup_count')
        .build()
    ),
    ('enrich_dropoff', CreateEnricher()
        .with_data(group_by='dropoff_node')
        .count_by(target='nodes', output_column='dropoff_count')
        .build()
    ),
    ("viz", StaticVisualiser())
])

In [ ]:
data, graph, nodes, edges = pipeline.compose_transform(
    latitude_column_name='pickup_latitude', # Not sure yet if this is highly necessary. Future releases could make this "non-mandatory-anymore" so that we'd not have to put them when stacking mappings and enrichers.
    longitude_column_name='pickup_longitude'
)

## Step 3: Visualise the Enriched Network

We use `InteractiveVisualiser` to display the enriched network, targeting nodes to show pickup and dropoff counts interactively.

> **Note**: Ensure Jupyter extensions are installed (see README).

In [ ]:
viz = pipeline.visualise(
    # result_columns=['pickup_count', 'dropoff_count'], # For InteractiveVisualiser
    result_columns="pickup_count",
    target='nodes'
)
viz

## Conclusion

Congratulations! 🎉 You’ve built an advanced urban pipeline with OSMNxMapping [Multiple Mappings and Enrichments], efficiently processing taxi trip data to map and enrich a Downtown Brooklyn network with pickup and dropoff counts. This workflow demonstrates the power of chaining multiple mappings and enrichments seamlessly.

For simpler examples, check 'Basic Pipelining'. For target exploration, see 'Advanced Pipelining [Exploring Targets]'. Explore the OSMNxMapping API docs and `examples/` for more!

Happy urban mapping! 🌆